In [3]:
import numpy as np
import pandas as pd
import pickle

In [4]:
p = './data/pretraining/raw/c.pkl'
data = pickle.load(open(p, "rb"))


In [7]:
data[0]

{'function': 'int _getdns_bindata_is_dname ( getdns_bindata * bindata ) { size_t i = 0 , n_labels = 0 ; while ( i < bindata -> size && bindata -> data [ i ] ) { if ( bindata -> data [ i ] & 0xC0 ) /* Compression pointer! */ return 0 ; i += ( ( size_t ) bindata -> data [ i ] ) + 1 ; n_labels ++ ; } if ( i < bindata -> size && ! bindata -> data [ i ] ) { n_labels ++ ; i ++ ; } return i == bindata -> size && n_labels > 1 && bindata -> data [ bindata -> size - 1 ] == 0 ; }'}

```c
int _getdns_bindata_is_dname ( getdns_bindata * bindata ) 
    { 
        size_t i = 0 , n_labels = 0 ; 
        while ( i < bindata -> size && bindata -> data [ i ] ) 
        { 
            if ( bindata -> data [ i ] & 0xC0 )
                /* Compression pointer! */ 
                return 0 ; 
            i += ( ( size_t ) bindata -> data [ i ] ) + 1 ; 
            n_labels ++ ; 
        } 
        if ( i < bindata -> size && ! bindata -> data [ i ] ) 
        { 
            n_labels ++ ; i ++ ; 
        } 
        return i == bindata -> size && n_labels > 1 && bindata -> data [ bindata -> size - 1 ] == 0 ; 
    }
```

In [5]:
data[1]

{'function': 'getdns_return_t getdns_dict_get_dict ( const getdns_dict * dict , const char * name , getdns_dict * * answer ) { getdns_return_t r ; getdns_item * item ; if ( ! dict || ! name || ! answer ) return GETDNS_RETURN_INVALID_PARAMETER ; if ( ( r = _getdns_dict_find ( dict , name , & item ) ) ) return r ; if ( item -> dtype != t_dict ) return GETDNS_RETURN_WRONG_TYPE_REQUESTED ; * answer = item -> data . dict ; return GETDNS_RETURN_GOOD ; }'}

```c
getdns_return_t getdns_dict_get_dict ( const getdns_dict *dict , const char *name , getdns_dict **answer ) 
{ 
    getdns_return_t r ; 
    getdns_item *item ; 
    if ( ! dict || ! name || ! answer ) 
        return GETDNS_RETURN_INVALID_PARAMETER ; 
    if ( ( r = _getdns_dict_find ( dict , name , & item ) ) ) 
        return r ; 
    if ( item -> dtype != t_dict ) 
        return GETDNS_RETURN_WRONG_TYPE_REQUESTED ; 
    *answer = item -> data . dict ; 
    return GETDNS_RETURN_GOOD ; 
}
```

In [8]:
import jsonlines

# Example 1, Dead Code Insertion
```json
{
    "source": "static struct monitor * FindScreen ( fscreen_scr_arg * arg , fscreen_scr_t screen ) { struct monitor * m = NULL ; fscreen_scr_arg tmp ; if ( monitor_get_count ( ) == 0 ) return ( monitor_by_name ( GLOBAL_SCREEN_NAME ) ) ; switch ( screen ) { case FSCREEN_GLOBAL : m = monitor_by_name ( GLOBAL_SCREEN_NAME ) ; break ; case FSCREEN_PRIMARY : case FSCREEN_CURRENT : if ( ! arg ) { tmp . mouse_ev = NULL ; arg = & tmp ; } GetMouseXY ( arg -> mouse_ev , & arg -> xypos . x , & arg -> xypos . y ) ; 

    for ( ; false ; ) { tmp . xypos . x = 0 ;}

    case FSCREEN_XYPOS : if ( ! arg ) { tmp . xypos . x = 0 ; tmp . xypos . y = 0 ; arg = & tmp ; } m = FindScreenOfXY ( arg -> xypos . x , arg -> xypos . y ) ; break ; case FSCREEN_BY_NAME : if ( arg == NULL || arg -> name == NULL ) { break ; } m = monitor_by_name ( arg -> name ) ; break ; default : break ; } return ( m ) ; }",

    "target": "static struct monitor * FindScreen ( fscreen_scr_arg * arg , fscreen_scr_t screen ) { struct monitor * m = NULL ; fscreen_scr_arg tmp ; if ( monitor_get_count ( ) == 0 ) return ( monitor_by_name ( GLOBAL_SCREEN_NAME ) ) ; switch ( screen ) { case FSCREEN_GLOBAL : m = monitor_by_name ( GLOBAL_SCREEN_NAME ) ; break ; case FSCREEN_PRIMARY : case FSCREEN_CURRENT : if ( ! arg ) { tmp . mouse_ev = NULL ; arg = & tmp ; } GetMouseXY ( arg -> mouse_ev , & arg -> xypos . x , & arg -> xypos . y ) ; 
    
    case FSCREEN_XYPOS : if ( ! arg ) { tmp . xypos . x = 0 ; tmp . xypos . y = 0 ; arg = & tmp ; } m = FindScreenOfXY ( arg -> xypos . x , arg -> xypos . y ) ; break ; case FSCREEN_BY_NAME : if ( arg == NULL || arg -> name == NULL ) { break ; } m = monitor_by_name ( arg -> name ) ; break ; default : break ; } return ( m ) ; }",

    "transformer": "DeadCodeInserter"
}
```

**The branches never reach**

**Expressions never used**

Detecting dead code is costly. Inserting some more complicated dead code

# Example 2, Variable Renaming
```json
{
    "source": "void MyXGrabKeyboard ( Display * VAR_0 ) { keyboard_grab_count ++ ; XGrabKeyboard ( VAR_0 , VAR_1 ( VAR_0 , DefaultScreen ( VAR_0 ) ) , False , GrabModeAsync , GrabModeAsync , CurrentTime ) ; return ; }", 
    
    "target": "void MyXGrabKeyboard ( Display * dpy ) { keyboard_grab_count ++ ; XGrabKeyboard ( dpy , RootWindow ( dpy , DefaultScreen ( dpy ) ) , False , GrabModeAsync , GrabModeAsync , CurrentTime ) ; return ; }", 
    
    "transformer": "VarRenamer"
}
```

Variable referencing error, mentioned in JIGSAW

# Example 3, Block Swap
```json
{
    "source": "char * concatenated_filename ( const char * directory , const char * filename , const char * suffix ) { char * result ; char * p ; 
    
    if ( strcmp ( directory , \".\" ) == 0 ) { result = ( char * ) malloc ( strlen ( filename ) + ( suffix != NULL ? strlen ( suffix ) : 0 ) + 1 ) ; if ( result == NULL ) return NULL ; p = result ; } 
    
    else 
    { size_t directory_len = strlen ( directory ) ; int need_slash = ( directory_len > FILE_SYSTEM_PREFIX_LEN ( directory ) && ! ISSLASH ( directory [ directory_len - 1 ] ) ) ; result = ( char * ) malloc ( directory_len + need_slash + strlen ( filename ) + ( suffix != NULL ? strlen ( suffix ) : 0 ) + 1 ) ; if ( result == NULL ) return NULL ; memcpy ( result , directory , directory_len ) ; p = result + directory_len ; if ( need_slash ) * p ++ = ' / ' ; } 
    
    p = stpcpy ( p , filename ) ; if ( suffix != NULL ) stpcpy ( p , suffix ) ; return result ; }",
    
    "target": "char * concatenated_filename ( const char * directory , const char * filename , const char * suffix ) { char * result ; char * p ; 
    
    
    if ( strcmp ( directory , \".\" ) != 0 ) 
    { size_t directory_len = strlen ( directory ) ; int need_slash = ( directory_len > FILE_SYSTEM_PREFIX_LEN ( directory ) && ! ISSLASH ( directory [ directory_len - 1 ] ) ) ; result = ( char * ) malloc ( directory_len + need_slash + strlen ( filename ) + ( suffix != NULL ? strlen ( suffix ) : 0 ) + 1 ) ; if ( result == NULL ) return NULL ; memcpy ( result , directory , directory_len ) ; p = result + directory_len ; if ( need_slash ) * p ++ = ' / ' ; }
    
    else 
    { result = ( char * ) malloc ( strlen ( filename ) + ( suffix != NULL ? strlen ( suffix ) : 0 ) + 1 ) ; if ( result == NULL ) return NULL ; p = result ; } 
    
    p = stpcpy ( p , filename ) ; if ( suffix != NULL ) stpcpy ( p , suffix ) ; return result ; }", 
    
    "transformer": "BlockSwap"
}
```

Can this be useful?

# Example 4, Operand Swap ?

```json
{
    "source": "void cpp_register_pragma_space ( pfile , space ) cpp_reader * pfile ; const char * space ; { struct pragma_entry * new ; const struct pragma_entry * p = pfile -> pragmas ; size_t len = strlen ( space ) ; while ( p ) { if ( p -> isnspace && 

    p -> len == len 
    
    && ! memcmp ( p -> name , space , len ) ) return ; p = p -> next ; } new = xnew ( struct pragma_entry ) ; new -> name = space ; new -> len = len ; new -> isnspace = 1 ; new -> u . space = 0 ; new -> next = pfile -> pragmas ; pfile -> pragmas = new ; }", 
    
    "target": "void cpp_register_pragma_space ( pfile , space ) cpp_reader * pfile ; const char * space ; { struct pragma_entry * new ; const struct pragma_entry * p = pfile -> pragmas ; size_t len = strlen ( space ) ; while ( p ) { if ( p -> isnspace && 
    
    len == p -> len 
    
    && ! memcmp ( p -> name , space , len ) ) return ; p = p -> next ; } new = xnew ( struct pragma_entry ) ; new -> name = space ; new -> len = len ; new -> isnspace = 1 ; new -> u . space = 0 ; new -> next = pfile -> pragmas ; pfile -> pragmas = new ; }", 
    
    "transformer": "OperandSwap"  
}
```

# Example 5, For-While Transforming ?

```json
{"source": "static void libdef_lua ( BuildCtx * ctx , char * p , int arg ) { UNUSED ( arg ) ; if ( ctx -> mode == BUILD_libdef ) { int i ; i = 0 ; 

while ( libbc_map [ i ] name != NULL ) 

{ if ( ! strcmp ( libbc_map [ i ] name , p ) ) { int ofs = libbc_map [ i ] ofs ; int len = libbc_map [ i + 1 ] ofs - ofs ; obuf [ 2 ] ++ ; * optr ++ = LIBINIT_LUA ; libdef_name ( p , 0 ) ; memcpy ( optr , libbc_code + ofs , len ) ; libdef_fixupbc ( optr ) ; optr += len ; i ++ ; return ; } i ++ ; } fprintf ( stderr , \"Error: missing libbc definition for %s\\ \" , p ) ; exit ( 1 ) ; } }", 

"target": "static void libdef_lua ( BuildCtx * ctx , char * p , int arg ) { UNUSED ( arg ) ; if ( ctx -> mode == BUILD_libdef ) { int i ; 

for ( i = 0 ; libbc_map [ i ] . name != NULL ; i ++ ) 

{ if ( ! strcmp ( libbc_map [ i ] . name , p ) ) { int ofs = libbc_map [ i ] . ofs ; int len = libbc_map [ i + 1 ] . ofs - ofs ; obuf [ 2 ] ++ ; * optr ++ = LIBINIT_LUA ; libdef_name ( p , 0 ) ; memcpy ( optr , libbc_code + ofs , len ) ; libdef_fixupbc ( optr ) ; optr += len ; return ; } } fprintf ( stderr , \"Error: missing libbc definition for %s\\ \" , p ) ; exit ( 1 ) ; } }", 

"transformer": "ForWhileTransformer"}
```

# Example 6, Confusion Removal

```json
{"source": "static void place_progress_buttons ( WDialog * h , gboolean suspended ) { 

    const size_t i = suspended ? 2 : 1 ; 

    Widget * w = WIDGET ( h ) ; int buttons_width ; buttons_width = 2 + progress_buttons [ 0 ] . len + progress_buttons [ 3 ] . len ; buttons_width += progress_buttons [ i ] . len ; button_set_text ( BUTTON ( progress_buttons [ i ] . w ) , progress_buttons [ i ] . text ) ; progress_buttons [ 0 ] . w -> x = w -> x + ( w -> cols - buttons_width ) / 2 ; progress_buttons [ i ] . w -> x = progress_buttons [ 0 ] . w -> x + progress_buttons [ 0 ] . len + 1 ; progress_buttons [ 3 ] . w -> x = progress_buttons [ i ] . w -> x + progress_buttons [ i ] . len + 1 ; }", 

"target": "static void place_progress_buttons ( WDialog * h , gboolean suspended ) { 
    
    const size_t i ; if ( suspended ) { i = 2 ; } else { i = 1 ; } 
    
    Widget * w = WIDGET ( h ) ; int buttons_width ; buttons_width = 2 + progress_buttons [ 0 ] len + progress_buttons [ 3 ] len ; buttons_width += progress_buttons [ i ] len ; button_set_text ( BUTTON ( progress_buttons [ i ] w ) , progress_buttons [ i ] text ) ; progress_buttons [ 0 ] w -> x = w -> x + ( w -> cols - buttons_width ) / 2 ; progress_buttons [ i ] w -> x = progress_buttons [ 0 ] w -> x + progress_buttons [ 0 ] len + 1 ; progress_buttons [ 3 ] w -> x = progress_buttons [ i ] w -> x + progress_buttons [ i ] len + 1 ; }", 

"transformer": "ConfusionRemover"}
```

Can we use the same transformations? 

Probably not

- Most of them will be [KEEP]
- 

Our goal? 

Code Generation Task

1
- Code Translation, Code -> Code, differente languages
- Code Refinement, Code -> Code, generation Post-editing, simplification
- Clone Detection, Pre-training taks could be removing clone, where masked by [DELETE] is clone

2
- Code Summarization, Code -> NL, some code might be reserved, 
- Code Generation, NL -> Code, some words might be reserved, 
- Code Search?, NL -> Code, some words might be reserved, 


**While seq2seq frameworks offer a generic tool for modeling almost any kind of text-to-text transduction, there are still many real-world tasks where generating target texts completely from scratch —- as is done with seq2seq approaches—can be unnecessary. This is especially true for monolingual settings where input and output texts have relatively high degrees of overlap. In such cases a natural approach is to cast conditional text generation as a text-editing task, where the model learns to recon- struct target texts by applying a set of edit operations to the inputs. Typically, the set of edit operations is fixed and pre-defined ahead of time, which on one hand limits the flexibility of the model to reconstruct arbitrary output texts from their inputs, but on the other leads to higher sample-efficiency as the limited set of allowed operations significantly reduces the search space.**

In [ ]:
import tensorflow as tf
import numpy as np
import IPython.display as display
filenames = '/content/felix/output/train.tfrecord'
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset
for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)    

# Felix dataset 1 example

```json
features {
  feature {
    key: "input_ids"
    value {
      int64_list {
        value: 101
        value: 9854
        value: 14670
        value: 1997
        value: 2635
        value: 2091
        value: 13258
        value: 6905
        value: 3136
        value: 2003
        value: 3697
        value: 1012
        value: 2130
        value: 2312
        value: 25430
        value: 15457
        value: 2099
        value: 7635
        value: 1012
        value: 102
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
      }
    }
  }
  feature {
    key: "input_mask"
    value {
      int64_list {
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 1
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
      }
    }
  }
  feature {
    key: "input_tokens"
    value {
      bytes_list {
        value: "[CLS]"
        value: "measuring"
        value: "impacts"
        value: "of"
        value: "taking"
        value: "down"
        value: "fare"
        value: "abuse"
        value: "operations"
        value: "is"
        value: "difficult"
        value: "."
        value: "even"
        value: "large"
        value: "sw"
        value: "##ipe"
        value: "##r"
        value: "rings"
        value: "."
        value: "[SEP]"
      }
    }
  }
  feature {
    key: "label_tokens"
    value {
      bytes_list {
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP|2"
        value: "DELETE"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
        value: "KEEP"
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 5
        value: 3
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 2
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
      }
    }
  }
  feature {
    key: "labels_mask"
    value {
      float_list {
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 6.666666507720947
        value: 6.666666507720947
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.37037035822868347
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
      }
    }
  }
  feature {
    key: "point_indexes"
    value {
      int64_list {
        value: 1
        value: 2
        value: 3
        value: 4
        value: 5
        value: 6
        value: 7
        value: 8
        value: 9
        value: 10
        value: 12
        value: 0
        value: 13
        value: 14
        value: 15
        value: 16
        value: 17
        value: 18
        value: 19
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
      }
    }
  }
  feature {
    key: "segment_ids"
    value {
      int64_list {
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
      }
    }
  }
  feature {
    key: "text_source"
    value {
      bytes_list {
        value: "measuring impacts of taking down fare abuse operations is difficult . even large swiper rings ."
      }
    }
  }
  feature {
    key: "text_target"
    value {
      bytes_list {
        value: "measuring impacts of taking down fare abuse operations is difficult , because even large swiper rings ."
      }
    }
  }
}
```

# Tring cursor

- bubble sort
- leetcode medium
- leetcode hard

![Pasted Graphic 6.png](./leetcode.jpg)